<a href="https://colab.research.google.com/github/SenaDay/Projeto_imersao_Alura_FrutaBoa/blob/main/Fruta_Boa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Instala e importa a lib da GenerativeAI

In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import io
import json
import mimetypes
import pathlib
import pprint
import requests

import PIL.Image
import IPython.display
from IPython.display import Markdown

### Importa a lib de acesso ao Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Set the API key

Obtém a API Key a partir dos dados do usuário

In [ ]:
from google.colab import userdata


API_KEY=userdata.get('API_KEY')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Converte os argumentos

In [ ]:
model = 'gemini-1.5-pro-latest' # @param {isTemplate: true}
contents_b64 = 'W3sicGFydHMiOlt7InRleHQiOiJSZXNwb3N0YXMgZGVzY29udHJhw61kYXMifSx7InRleHQiOiJJbWFnZW06ICJ9LHsidGV4dCI6IkRlc2NyacOnw6NvOiBFc3NhIGxhcmFuamEgdMOhIHRvcCEgQ29yIHZpYnJhbnRlLCBmaXJtaW5oYSBhbyB0b3F1ZSBlIHVtIGNoZWlyaW5obyBxdWUgasOhIGTDoSDDoWd1YSBuYSBib2NhLiDDiSBzw7MgbW9yZGVyIGUgc2UgZGVsaWNpYXIhIn0seyJ0ZXh0IjoiSW1hZ2VtOiAifSx7InRleHQiOiJEZXNjcmnDp8OjbzogRXNzYSBsYXJhbmphIGHDrSBqw6EgZXJhLCBtZWxob3IgZGVpeGFyIGRlIGxhZG8hIFTDoSBtdXJjaGEsIGEgY2FzY2EgdMOhIG1vbGUgZSwgcGlvciBhaW5kYSwgc2Ugcm9sYXIgdW0gY2hlaXJvIG1laW8gYXplZG8sIGHDrSDDqSBxdWUgbsOjbyBkw6EgbWVzbW8hIE1hcyByZWxheGEsIHVtYSBsYXJhbmphIGJvYSB0w6EgY29tIGEgY2FzY2EgZmlybWUsIHVtIGxhcmFuamEgYmVtIGFsYXJhbmphZG8sIGUgcXVhbmRvIHZvY8OqIGFwZXJ0YSwgc2VudGUgZmlybWV6YSwgc2FiZT8gQWgsIGUgYXF1ZWxlIGNoZWlyaW5obyBnb3N0b3NvLCBxdWUgc8OzIGRlIHNlbnRpciBqw6EgYWJyZSBvIGFwZXRpdGUuIEVzc2Egw6kgYSBwZWRpZGEgY2VydGEhIn0seyJ0ZXh0IjoiSW1hZ2VtOiAifSx7InRleHQiOiJEZXNjcmnDp8OjbzogRXNzZSBraXdpIGrDoSB0w6EgbmFxdWVsYSB2aWJlICdwYXNzb3UgZG8gcG9udG8nLCB0w6EgbW9sZSBkZW1haXMsIGEgY2FzY2EgdMOhIGVucnVnYWRpbmhhLiBFc3F1ZWNlISBBZ29yYSwgdW0ga2l3aSBmcmVzcXVpbmhvIMOpIG91dHJhIGhpc3TDs3JpYTogYSBjYXNjYSB0w6EgZmlybWUsIGNvbSB1bSBsZXZlIHRvcXVlIGRlIHBlbGluaG9zLCBlIHF1YW5kbyB2b2PDqiBhcGVydGEsIHNlbnRlIHVtIHBvdWNvIGRlIGZpcm1lemEsIG1hcyBhaW5kYSBjZWRlIHVtIHBvdWNvLiDDiSBkaXNzbyBxdWUgZXN0YW1vcyBmYWxhbmRvISJ9LHsidGV4dCI6IkltYWdlbTogIn0seyJ0ZXh0IjoiRGVzY3Jpw6fDo286IEVzc2EgY2Vub3VyYSB0w6Egbm8gcG9udG8gY2VydG8hIE9saGEgc8OzLCB0w6EgZmlybWUsIGNvbSB1bWEgY29yIHZpYnJhbnRlIGUgYSBjYXNjYSBsaXNhLCBzZW0gbWFuY2hhcy4ifSx7InRleHQiOiJJbWFnZW06ICJ9LHsidGV4dCI6IkRlc2NyacOnw6NvOiAifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5Miwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [ ]:
## Conteúdos de referência

In [ ]:
## Configuração de como serão as respostas da AI

In [ ]:
## Configurações de segurança para os resultados

### Carrega os arquivos de imagem do drive

In [ ]:
for content in contents:
  for n, part in enumerate(content['parts']):
    if image:=part.get('image', None):
      if drive_id:=image.get('drive_id', None):
        path = next(pathlib.Path(f'/gdrive/.shortcut-targets-by-id/{drive_id}').glob('*'))
        data = path.read_bytes()
        mime_type, _ = mimetypes.guess_type(path)
      elif image_url:=image.get('image_url', None):
        response = requests.get(image_url)
        data = response.content
        mime_type = response.headers['content-type']
      else:
        raise ValueError('Either drive_id or image_url must be provided.')

      if mime_type is None:
        # Guess!
        mime_type = 'image/png'

      blob = {'data': data, 'mime_type': mime_type}
      content['parts'][n] = {'inline_data': blob}

In [ ]:
import IPython.display
import PIL.Image
import io

for content in contents:
    for part in content['parts']:
        if text := part.get('text', None):
            print(text)
        elif data := part.get('inline_data', None):
            img = PIL.Image.open(io.BytesIO(data['data']))
            img.thumbnail([512,512])
            IPython.display.display(img)
    print('_'*80)


Respostas descontraídas
Imagem: 
Descrição: Essa laranja tá top! Cor vibrante, firminha ao toque e um cheirinho que já dá água na boca. É só morder e se deliciar!
Imagem: 
Descrição: Essa laranja aí já era, melhor deixar de lado! Tá murcha, a casca tá mole e, pior ainda, se rolar um cheiro meio azedo, aí é que não dá mesmo! Mas relaxa, uma laranja boa tá com a casca firme, um laranja bem alaranjado, e quando você aperta, sente firmeza, sabe? Ah, e aquele cheirinho gostoso, que só de sentir já abre o apetite. Essa é a pedida certa!
Imagem: 
Descrição: Esse kiwi já tá naquela vibe 'passou do ponto', tá mole demais, a casca tá enrugadinha. Esquece! Agora, um kiwi fresquinho é outra história: a casca tá firme, com um leve toque de pelinhos, e quando você aperta, sente um pouco de firmeza, mas ainda cede um pouco. É disso que estamos falando!
Imagem: 
Descrição: Essa cenoura tá no ponto certo! Olha só, tá firme, com uma cor vibrante e a casca lisa, sem manchas.
Imagem: 
Descrição: 
________

### Chama a API

In [ ]:
# Chama o modelo e imprime a resposta.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False)

In [ ]:
if generation_config.get('candidate_count', 1) == 1:
  display(Markdown(response.text))
else:
  print(response.candidates)

Mandou bem! 👍 Essas descrições tão show de bola! Bem descoladas e fáceis de entender, do jeito que a galera gosta. 😉 

Se quiser, pode mandar mais frutas e legumes pra gente dar uma olhada. Adoro ajudar a galera a escolher as melhores! 😄 


In [ ]:
response.candidates

[content {
  parts {
    text: "Mandou bem! \360\237\221\215 Essas descri\303\247\303\265es t\303\243o show de bola! Bem descoladas e f\303\241ceis de entender, do jeito que a galera gosta. \360\237\230\211 \n\nSe quiser, pode mandar mais frutas e legumes pra gente dar uma olhada. Adoro ajudar a galera a escolher as melhores! \360\237\230\204 \n"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

In [ ]:
response.prompt_feedback